# Auction Playground

AuctionGym has several configuration options that detail the type of auction, how bidders behave. We will look at different bidder configurations and their impact on their own welfare, surplus and revenue. 

All you need to do is run the 4 simple steps below.

**Note: Make sure to install the requirements using a virtual environment before booting up the notebook using instructions below**

- python3 -m venv gympy
- source gympy/bin/activate
- jupyter notebook

## 🚧 Step 1: Basic setup

In [ ]:

from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from main import parse_config, instantiate_agents, instantiate_auction
from tqdm.notebook import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib notebook
# Parse default configuration file
rng, config, agent_configs, agents2items, agents2item_values,\
num_runs, max_slots, embedding_size, embedding_var,\
obs_embedding_size = parse_config('../config/Mixed_FP_Oracle.json')

# obs_embedding_size = parse_config('../config/SP_Oracle.json')
config

## ⚙️ Step 2: Interactively update auction configuration

Run the cell below and you will see sliders and drop downs to make the auction your own :) 

*P.S: It doesn't let you override all the configs interactively yet. Feel free to update the configs directly for the ones that are not supported yet*

In [ ]:
def auction_configurator(num_iter, num_participants_per_round, rounds_per_iter, allocation, bidder):
    config['num_iter'] = num_iter
    config['num_participants_per_round'] = num_participants_per_round
    config['rounds_per_iter'] = rounds_per_iter
    config['allocation'] = allocation
    config['bidder'] = bidder
interact(
    auction_configurator,
    num_iter=widgets.IntSlider(min=1, max=30, step=1, value=config['num_iter']),
    num_participants_per_round=widgets.IntSlider(min=1, max=6, step=1, value=config['num_participants_per_round']),
    rounds_per_iter=widgets.IntSlider(min=1, max=30000, step=1, value=config['rounds_per_iter']),
    allocation=['FirstPrice', 'SecondPrice'],
    bidder=['ValueLearningBidder', 'EmpiricalShadedBidder', 'TruthfulBidder', 'PolicyLearningBidder', 'DoublyRobustBidder']
);

## 🎉 Step 3: Run the auction 

*Note that it takes about 2-5 mins to run(mainly because some of the agents learns at the end of every iteration with a deep network). But once it runs, you can interactively explore the results.*

*If the runs take too long, try reducing the number settings "num_iter" or "rounds_per_iter" to reduce the number of auction runs.*

In [ ]:
agent_gross_utility_history = defaultdict(list)
agent_net_utility_history = defaultdict(list)
auction_revenue = []
social_welfare = []
social_surplus = []
fig = plt.figure()
fig, axs = plt.subplots(3, 1, constrained_layout=True)
fig.suptitle('Auction Metrics', fontsize=16)

axs[0].plot(auction_revenue)
axs[0].set_title('Auction Revenue')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('Revenue')

axs[1].plot(social_welfare)
axs[1].set_title('Social Welfare')
axs[1].set_xlabel('Iterations')
axs[1].set_ylabel('Welfare')


axs[2].plot(social_surplus)
axs[2].set_title('Social Surplus')
axs[2].set_xlabel('Iterations')
axs[2].set_ylabel('Surplus')
plt.ion()

fig.show()
fig.canvas.draw()

def run_auction():
    # Instantiate Agent and Auction objects
    agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)
    # Instantiate Auction object
    auction, num_iter, rounds_per_iter, output_dir =\
        instantiate_auction(rng,
                            config,
                            agents2items,
                            agents2item_values,
                            agents,
                            max_slots,
                            embedding_size,
                            embedding_var,
                            obs_embedding_size)
    for i in tqdm(range(num_iter)):
        for _ in range(rounds_per_iter):
            auction.simulate_opportunity()
        # Log 'Gross utility' or welfare
        social_welfare.append(sum([agent.gross_utility for agent in auction.agents]))
        # Log 'Net utility' or surplus
        social_surplus.append(sum([agent.net_utility for agent in auction.agents]))
        
        for agent_id, agent in enumerate(auction.agents):
            print(f'Updating agent {agent_id} for iteration {i}')
            agent.update(iteration=i)
            agent_gross_utility_history[agent.name].append(np.sum(agent.gross_utility_trends))
            agent_net_utility_history[agent.name].append(np.sum(agent.net_utility_trends))
            agent.clear_utility()
            pass
        
        # Log revenue
        auction_revenue.append(auction.revenue)
        auction.clear_revenue()
        
        # Interactive visualization
        axs[0].clear()
        axs[1].clear()
        axs[0].plot(auction_revenue)
        axs[0].set_title('Auction Revenue')
        axs[0].set_xlabel('Iterations')
        axs[0].set_ylabel('Revenue')
        axs[1].plot(social_welfare)
        axs[1].set_title('Social Welfare')
        axs[1].set_xlabel('Iterations')
        axs[1].set_ylabel('Welfare')
        axs[2].plot(social_surplus)
        axs[2].set_title('Social Surplus')
        axs[2].set_xlabel('Iterations')
        axs[2].set_ylabel('Surplus')
        fig.canvas.draw()
        
        
    print(f'Number of iterations: {num_iter}. Number of rounds per iteration {rounds_per_iter}')
    print(f'Total opportunities: {num_iter * rounds_per_iter}')
    print(f'Total agents: {len(auction.agents)}')
    return auction
auction = run_auction()
plt.close()

## 🔮 Step 4: Analyze the results

In [ ]:
plt.figure()
[plt.plot(agent_gross_utility_history[a.name], label = f'Gross Utility for {a.name}') for a in auction.agents]
plt.legend(loc="best");

In [ ]:
plt.figure()
[plt.plot(agent_net_utility_history[a.name], label = f'Net Utility for {a.name}') for a in auction.agents]
plt.legend(loc="best");

In [ ]:
plt.figure()
plt.plot(auction_revenue, label='Overall Auction Revenue')
plt.plot(social_welfare, label='Overall Social Welfare')
plt.plot(social_surplus, label='Overall Social Surplus')
plt.legend(loc="best");